# Delta Lake Transaction Logs - Interactive Demo

Welcome! This demo will teach you how Delta Lake's transaction log enables powerful features like ACID transactions and time travel.

---

## 🎯 What is Delta Lake?

**Delta Lake** is an open-source storage layer that brings ACID transactions to data lakes.

**Key innovation:** The **Transaction Log** (also called the **Delta Log**)

---

## 📖 What is the Transaction Log?

The transaction log is a **ordered record of every transaction** made to a Delta table.

**Think of it like:**
* A ledger in accounting - every change is recorded
* Git history for your data - every commit is tracked
* A journal - chronological record of all operations

**What it stores:**
* Which files were added
* Which files were removed
* Schema changes
* Table properties
* Metadata about each operation

---

## ✨ Why Does This Matter?

The transaction log enables:

1. **ACID Transactions** - Reliable, consistent data operations
2. **Time Travel** - Query historical versions of data
3. **Audit Trail** - See who changed what and when
4. **Concurrent Writes** - Multiple writers without conflicts
5. **Schema Evolution** - Track schema changes over time
6. **Rollback** - Undo mistakes by reverting to previous versions

---

## 🎯 What You'll Learn

1. **How transaction logs work** - Conceptual understanding
2. **DESCRIBE HISTORY** - View the transaction log
3. **ACID properties** - See them in action
4. **Time Travel** - Query historical data
5. **Practical use cases** - Real-world applications

**Let's get started!** 🚀

## 1. How Transaction Logs Work 📖

**High-level architecture:**

```
Delta Table
├── _delta_log/              (Transaction log directory)
│   ├── 00000000000.json    (Transaction 0 - CREATE TABLE)
│   ├── 00000000001.json    (Transaction 1 - INSERT)
│   ├── 00000000002.json    (Transaction 2 - UPDATE)
│   ├── 00000000003.json    (Transaction 3 - DELETE)
│   └── ...
└── data/                   (Parquet data files)
    ├── part-00000.parquet
    ├── part-00001.parquet
    └── ...
```

---

## 📝 What's in a Transaction Log Entry?

Each JSON file contains:

**1. Add actions** - Files added to the table
```json
{
  "add": {
    "path": "part-00000.parquet",
    "size": 1024,
    "modificationTime": 1234567890,
    "dataChange": true
  }
}
```

**2. Remove actions** - Files removed from the table
```json
{
  "remove": {
    "path": "part-00000.parquet",
    "deletionTimestamp": 1234567890
  }
}
```

**3. Metadata** - Schema and table properties
```json
{
  "metaData": {
    "schemaString": "...",
    "partitionColumns": [...],
    "configuration": {...}
  }
}
```

**4. Protocol** - Delta Lake version

**5. Commit info** - Who, when, what operation

### 📚 How Delta Lake Reads Work

**When you query a Delta table:**

1. **Read the transaction log** - Start from the latest transaction
2. **Reconstruct table state** - Replay all transactions to know which files are current
3. **Read only current files** - Skip deleted/old files
4. **Return results** - Query only the active data files

**Example:**

```
Transaction 0: ADD file1.parquet
Transaction 1: ADD file2.parquet
Transaction 2: REMOVE file1.parquet, ADD file3.parquet

Current state: file2.parquet, file3.parquet  ← Only these are read!
```

**Benefits:**
* ✅ Always consistent view of data
* ✅ No need to physically delete files immediately
* ✅ Enables time travel (read old transactions)
* ✅ Multiple readers see consistent snapshots

### ✍️ How Delta Lake Writes Work

**When you write to a Delta table:**

1. **Write data files** - Write new Parquet files to storage
2. **Create transaction entry** - Prepare JSON with add/remove actions
3. **Atomic commit** - Write transaction JSON with next sequence number
4. **Success or fail** - Either entire transaction succeeds or fails

**Example: INSERT operation**

```
Step 1: Write part-00005.parquet (new data)
Step 2: Create transaction JSON:
        {
          "add": {"path": "part-00005.parquet", ...}
        }
Step 3: Atomically write 00000000005.json
Step 4: Transaction committed! ✅
```

**Atomicity guarantee:**
* If step 3 fails, the data file exists but is ignored (orphan file)
* If step 3 succeeds, the transaction is permanent
* No partial transactions - all or nothing!

**Optimistic concurrency:**
* Multiple writers can work simultaneously
* Conflicts detected at commit time
* Failed commits retry automatically

## ✨ Transaction Log Benefits

### **1. ACID Transactions**
* **Atomicity** - All or nothing commits
* **Consistency** - Data always in valid state
* **Isolation** - Concurrent operations don't interfere
* **Durability** - Committed data is permanent

### **2. Time Travel**
* Query data as it existed at any point in time
* Rollback mistakes
* Audit data changes
* Compare versions

### **3. Schema Evolution**
* Track schema changes over time
* Add/remove columns safely
* Maintain backward compatibility

### **4. Metadata Operations**
* Fast metadata queries (no data scan)
* Efficient file pruning
* Statistics for optimization

### **5. Concurrent Access**
* Multiple readers always see consistent data
* Multiple writers with conflict resolution
* No locking required for reads

### **6. Audit Trail**
* Who made changes
* When changes occurred
* What operation was performed
* Complete history of table evolution

## 2. See Transaction Logs in Action 🎬

Let's create a Delta table and perform operations to see how each creates a transaction log entry.

**We'll use DESCRIBE HISTORY to view the transaction log** - no file system access needed!

In [0]:
%sql
-- Create a new Delta table
-- This creates Transaction 0 in the log

CREATE OR REPLACE TABLE main.default.products_demo (
  product_id INT,
  product_name STRING,
  category STRING,
  price DOUBLE,
  stock_quantity INT
)
USING DELTA
COMMENT 'Demo table for transaction log exploration'

In [0]:
%sql
-- DESCRIBE HISTORY shows the transaction log!
-- This is how we view transactions without file system access

DESCRIBE HISTORY main.default.products_demo

In [0]:
%sql
-- Insert some data
-- This creates a new transaction (Transaction 1)

INSERT INTO main.default.products_demo VALUES
  (1, 'Laptop', 'Electronics', 999.99, 50),
  (2, 'Mouse', 'Electronics', 29.99, 200),
  (3, 'Desk', 'Furniture', 299.99, 30),
  (4, 'Chair', 'Furniture', 199.99, 45),
  (5, 'Monitor', 'Electronics', 399.99, 75)

In [0]:
%sql
-- Now we should see 2 transactions:
-- Version 0: CREATE TABLE
-- Version 1: INSERT

DESCRIBE HISTORY main.default.products_demo

In [0]:
%sql
-- Update prices (discount on electronics)
-- This creates Transaction 2

UPDATE main.default.products_demo
SET price = price * 0.9
WHERE category = 'Electronics'

In [0]:
%sql
-- Now we have 3 transactions
-- Notice the operation type and timestamp

DESCRIBE HISTORY main.default.products_demo

In [0]:
%sql
-- Delete out-of-stock items
-- This creates Transaction 3

DELETE FROM main.default.products_demo
WHERE stock_quantity < 40

In [0]:
%sql
-- Now we have 4 transactions
-- Each operation is recorded!

DESCRIBE HISTORY main.default.products_demo

In [0]:
%sql
-- Add a new column to the table
-- This creates Transaction 4 with schema change

ALTER TABLE main.default.products_demo
ADD COLUMN supplier STRING

In [0]:
%sql
-- Transaction log tracks schema evolution too!

DESCRIBE HISTORY main.default.products_demo

### 🔍 Understanding DESCRIBE HISTORY Output

**Key columns in the transaction log:**

* **version** - Transaction number (0, 1, 2, 3...)
* **timestamp** - When the transaction occurred
* **userId** - Who made the change
* **userName** - User's email/name
* **operation** - Type of operation (CREATE, INSERT, UPDATE, DELETE, etc.)
* **operationParameters** - Details about the operation
* **readVersion** - Version read before this write
* **isolationLevel** - Transaction isolation level
* **isBlindAppend** - Whether operation only added data

**Common operations you'll see:**
* `CREATE TABLE` / `CREATE OR REPLACE TABLE`
* `WRITE` / `INSERT`
* `UPDATE`
* `DELETE`
* `MERGE`
* `OPTIMIZE`
* `VACUUM`
* `SET TBLPROPERTIES`
* `ADD COLUMNS`

**💡 Key insight:** Every operation creates a new version in the transaction log!

## 3. ACID Properties in Action 🛡️

The transaction log enables **ACID guarantees** - let's see each property in action!

### ⚛️ Atomicity - All or Nothing

**What is Atomicity?**

A transaction either **completely succeeds** or **completely fails** - no partial updates.

**How transaction log enables this:**
* Data files are written first
* Transaction JSON is written atomically (last step)
* If JSON write fails, data files are ignored (orphaned)
* If JSON write succeeds, entire transaction is committed

**Example scenario:**
```sql
-- Insert 1 million rows
INSERT INTO table SELECT * FROM large_dataset
```

**Without Delta Lake:**
* If it fails halfway, you have 500K rows (partial data) ❌
* Data is corrupted
* Need to manually clean up

**With Delta Lake:**
* If it fails, transaction log entry is not written
* Table still has 0 rows (no partial data) ✅
* Automatic rollback
* No cleanup needed

**💡 Key Point:** The transaction log entry is the "commit" - without it, data doesn't exist!

In [0]:
%sql
-- Let's insert multiple rows in one transaction
-- Either ALL 3 rows are inserted, or NONE are

INSERT INTO main.default.products_demo VALUES
  (6, 'Keyboard', 'Electronics', 79.99, 100),
  (7, 'Webcam', 'Electronics', 89.99, 60),
  (8, 'Headphones', 'Electronics', 149.99, 80);

-- Check the data
SELECT * FROM main.default.products_demo

In [0]:
%sql
-- This INSERT created ONE transaction
-- All 3 rows are part of the same atomic commit

DESCRIBE HISTORY main.default.products_demo
LIMIT 5

### ⚖️ Consistency - Always Valid State

**What is Consistency?**

Data always moves from one **valid state** to another valid state.

**How transaction log enables this:**
* Schema is enforced at write time
* Constraints are validated
* Invalid data is rejected before commit
* Transaction log only records valid transactions

**Example:**
```sql
-- This will FAIL - violates schema
INSERT INTO products_demo VALUES ('invalid', 'Product', 'Cat', 'not_a_number', 10)
```

**Result:**
* Transaction is rejected ❌
* No transaction log entry created
* Table remains in valid state ✅
* No cleanup needed

**💡 Key Point:** Transaction log only contains valid, consistent transactions!

### 🔒 Isolation - Concurrent Operations

**What is Isolation?**

Concurrent transactions don't interfere with each other.

**How transaction log enables this:**
* Each transaction gets a unique version number
* Readers see a consistent snapshot (specific version)
* Writers use optimistic concurrency control
* Conflicts detected at commit time

**Scenario: Two users writing simultaneously**

```
User A:                          User B:
Read version 5                   Read version 5
Modify data                      Modify data
Write files                      Write files
Try to commit version 6          Try to commit version 6
  ✅ Success! (first to commit)    ❌ Conflict detected!
                                 Retry with version 6
                                 Commit as version 7 ✅
```

**Benefits:**
* No locking required for reads
* Multiple readers always see consistent data
* Writers automatically retry on conflict
* No data corruption from concurrent writes

**💡 Key Point:** Transaction log sequence numbers prevent conflicts!

### 💾 Durability - Permanent Changes

**What is Durability?**

Once a transaction is committed, it's **permanent** - survives failures.

**How transaction log enables this:**
* Transaction log is written to durable storage (S3, ADLS, GCS)
* Cloud storage provides durability guarantees
* Once JSON file is written, transaction is permanent
* Can reconstruct table state from transaction log

**Failure scenarios:**

**Scenario 1: Cluster crashes during write**
* Data files written: ✅
* Transaction log NOT written: ❌
* Result: Transaction not committed, data ignored
* Table remains in previous valid state

**Scenario 2: Cluster crashes after commit**
* Data files written: ✅
* Transaction log written: ✅
* Result: Transaction committed and permanent
* New cluster can read the data immediately

**💡 Key Point:** Transaction log in cloud storage = durable commits!

## 4. Time Travel ⏰

**What is Time Travel?**

Time Travel lets you query **historical versions** of your Delta table using the transaction log.

**How it works:**
* Transaction log records every version
* Each version is a snapshot of the table at that point in time
* You can query any previous version
* Data files are retained (until VACUUM)

**Use cases:**
* Audit data changes
* Recover from mistakes
* Compare versions
* Reproduce reports
* Debug data issues

Let's see it in action!

In [0]:
%sql
-- First, let's see the current state
SELECT * FROM main.default.products_demo
ORDER BY product_id

In [0]:
%sql
-- Review all the versions we created
DESCRIBE HISTORY main.default.products_demo

In [0]:
%sql
-- Query Version 1 (right after initial INSERT, before UPDATE)
-- Use VERSION AS OF syntax

SELECT * FROM main.default.products_demo VERSION AS OF 1
ORDER BY product_id

In [0]:
%sql
-- Compare current version with Version 1
-- Notice the prices changed (we applied 10% discount in UPDATE)

SELECT 
  'Version 1 (Before Update)' AS version,
  product_id,
  product_name,
  price
FROM main.default.products_demo VERSION AS OF 1

UNION ALL

SELECT 
  'Current (After Update)' AS version,
  product_id,
  product_name,
  price
FROM main.default.products_demo

ORDER BY product_id, version

In [0]:
%sql
-- You can also query by timestamp
-- Use TIMESTAMP AS OF syntax

SELECT * 
FROM main.default.products_demo 
TIMESTAMP AS OF '2024-01-01'
ORDER BY product_id

### 📚 Time Travel Syntax

**Query by version number:**
```sql
SELECT * FROM table VERSION AS OF 5
SELECT * FROM table@v5
```

**Query by timestamp:**
```sql
SELECT * FROM table TIMESTAMP AS OF '2024-01-15 10:30:00'
SELECT * FROM table@20240115
```

**Query by date:**
```sql
SELECT * FROM table TIMESTAMP AS OF '2024-01-15'
```

**In Python:**
```python
# By version
df = spark.read.format("delta").option("versionAsOf", 5).table("table")

# By timestamp
df = spark.read.format("delta").option("timestampAsOf", "2024-01-15").table("table")
```

**⚠️ Note:** Time travel only works for versions that haven't been vacuumed!

In [0]:
%sql
-- You can restore a table to a previous version
-- This creates a new transaction that reverts changes

RESTORE TABLE main.default.products_demo TO VERSION AS OF 1

In [0]:
%sql
-- Check the data - should be back to Version 1 state
-- Prices should be original (before discount)

SELECT * FROM main.default.products_demo
ORDER BY product_id

In [0]:
%sql
-- RESTORE creates a new transaction!
-- The old versions are still in the log

DESCRIBE HISTORY main.default.products_demo

### 🎯 Time Travel Use Cases

**1. Audit and Compliance**
```sql
-- See data as it existed on a specific date
SELECT * FROM sales TIMESTAMP AS OF '2024-12-31'
```
Perfect for regulatory reporting and audits.

**2. Recover from Mistakes**
```sql
-- Oops, deleted wrong data!
RESTORE TABLE my_table TO VERSION AS OF 10
```
Undo accidental deletes or bad updates.

**3. Reproduce Reports**
```sql
-- Reproduce last month's report exactly
SELECT * FROM metrics TIMESTAMP AS OF '2024-01-31'
```
Ensure report consistency.

**4. Debug Data Issues**
```sql
-- When did this value change?
SELECT * FROM table VERSION AS OF 5
UNION ALL
SELECT * FROM table VERSION AS OF 6
```
Track down when data changed.

**5. A/B Testing**
```sql
-- Compare algorithm results
SELECT * FROM predictions VERSION AS OF 10  -- Old model
SELECT * FROM predictions VERSION AS OF 15  -- New model
```
Compare different versions of data.

**6. Data Rollback**
```sql
-- Roll back to before bad ETL job
RESTORE TABLE my_table TO TIMESTAMP AS OF '2024-01-15 09:00:00'
```
Recover from pipeline failures.

## 5. Practical Patterns 🎯

Real-world patterns using transaction logs.

In [0]:
%sql
-- Create an audit report showing all changes to the table
-- This is perfect for compliance and debugging

SELECT 
  version,
  timestamp,
  operation,
  operationParameters,
  userName,
  CAST(operationMetrics.numOutputRows AS INT) AS rows_affected
FROM (
  DESCRIBE HISTORY main.default.products_demo
)
ORDER BY version DESC

In [0]:
%sql
-- Find when a specific product's price changed
-- Compare consecutive versions

WITH version_1 AS (
  SELECT product_id, product_name, price AS price_v1
  FROM main.default.products_demo VERSION AS OF 1
),
version_2 AS (
  SELECT product_id, product_name, price AS price_v2
  FROM main.default.products_demo VERSION AS OF 2
)
SELECT 
  v1.product_id,
  v1.product_name,
  v1.price_v1 AS original_price,
  v2.price_v2 AS updated_price,
  ROUND(v2.price_v2 - v1.price_v1, 2) AS price_change,
  ROUND((v2.price_v2 - v1.price_v1) * 100.0 / v1.price_v1, 2) AS percent_change
FROM version_1 v1
JOIN version_2 v2 ON v1.product_id = v2.product_id
WHERE v1.price_v1 != v2.price_v2

In [0]:
%sql
-- Check how long data has been retained
-- Useful for planning VACUUM operations

SELECT 
  MIN(timestamp) AS oldest_version,
  MAX(timestamp) AS newest_version,
  COUNT(*) AS total_versions,
  DATEDIFF(MAX(timestamp), MIN(timestamp)) AS retention_days
FROM (
  DESCRIBE HISTORY main.default.products_demo
)

In [0]:
%sql
-- Analyze what operations have been performed
-- Understand table usage patterns

SELECT 
  operation,
  COUNT(*) AS operation_count,
  MIN(timestamp) AS first_occurrence,
  MAX(timestamp) AS last_occurrence
FROM (
  DESCRIBE HISTORY main.default.products_demo
)
GROUP BY operation
ORDER BY operation_count DESC

## ⚠️ Transaction Log Limitations

**1. Storage Costs**
* Transaction log files accumulate over time
* Old data files are retained for time travel
* Use VACUUM to clean up old files

**2. VACUUM Removes History**
```sql
-- VACUUM deletes files older than retention period (default 7 days)
VACUUM table RETAIN 168 HOURS  -- 7 days
```
* After VACUUM, time travel to old versions fails
* Balance retention needs vs storage costs

**3. Performance Considerations**
* Very long transaction logs can slow down reads
* Use OPTIMIZE to compact files
* Consider checkpoint files (automatic every 10 commits)

**4. Serverless Limitations**
* Cannot directly access _delta_log files
* Use DESCRIBE HISTORY instead
* Use Delta Lake APIs for programmatic access

**5. Retention Limits**
* Default retention: 30 days
* Configurable with table properties
* Consider compliance requirements

## ✅ Transaction Log Best Practices

### **1. Monitoring**

✅ **Regularly check DESCRIBE HISTORY** - Monitor table changes  
✅ **Track operation metrics** - Understand data volumes  
✅ **Set up alerts** - Detect unexpected operations  

```sql
DESCRIBE HISTORY table
```

### **2. Retention Management**

✅ **Set appropriate retention** - Balance needs vs costs  
✅ **Document retention policy** - Compliance requirements  
✅ **Schedule VACUUM** - Clean up old files regularly  

```sql
-- Set retention period
ALTER TABLE table SET TBLPROPERTIES (
  'delta.logRetentionDuration' = '30 days',
  'delta.deletedFileRetentionDuration' = '7 days'
)

-- Vacuum old files
VACUUM table RETAIN 168 HOURS
```

### **3. Time Travel Usage**

✅ **Use for auditing** - Track data lineage  
✅ **Test before RESTORE** - Verify version before restoring  
✅ **Document version numbers** - For important snapshots  

```sql
-- Always check before restore
SELECT * FROM table VERSION AS OF 10 LIMIT 10
RESTORE TABLE table TO VERSION AS OF 10
```

### **4. Performance**

✅ **Run OPTIMIZE regularly** - Compact small files  
✅ **Use Z-ORDER** - Optimize for query patterns  
✅ **Monitor transaction log size** - Checkpoint files help  

```sql
OPTIMIZE table
OPTIMIZE table ZORDER BY (column)
```

### **5. Schema Evolution**

✅ **Track schema changes** - Review DESCRIBE HISTORY  
✅ **Test schema changes** - In dev before production  
✅ **Use mergeSchema carefully** - Understand implications  

```sql
-- Review schema changes
SELECT version, operation, operationParameters
FROM (DESCRIBE HISTORY table)
WHERE operation LIKE '%COLUMN%'
```

## 📚 Quick Reference

### **View Transaction Log**
```sql
DESCRIBE HISTORY table
DESCRIBE HISTORY table LIMIT 10
```

### **Time Travel Queries**
```sql
-- By version
SELECT * FROM table VERSION AS OF 5
SELECT * FROM table@v5

-- By timestamp
SELECT * FROM table TIMESTAMP AS OF '2024-01-15'
SELECT * FROM table@20240115
```

### **Restore Table**
```sql
RESTORE TABLE table TO VERSION AS OF 10
RESTORE TABLE table TO TIMESTAMP AS OF '2024-01-15'
```

### **Table Details**
```sql
DESCRIBE DETAIL table  -- Current state
DESCRIBE EXTENDED table  -- Full metadata
```

### **Maintenance**
```sql
-- Optimize
OPTIMIZE table
OPTIMIZE table ZORDER BY (column)

-- Vacuum
VACUUM table  -- Default 7 days retention
VACUUM table RETAIN 168 HOURS
VACUUM table DRY RUN  -- Preview what would be deleted
```

### **Table Properties**
```sql
-- Set retention
ALTER TABLE table SET TBLPROPERTIES (
  'delta.logRetentionDuration' = '30 days',
  'delta.deletedFileRetentionDuration' = '7 days'
)
```

## 💡 Key Concepts Summary

### **The Transaction Log**

1. **Ordered journal** of all changes to a Delta table
2. **JSON files** numbered sequentially (00000000000.json, 00000000001.json, ...)
3. **Records** add/remove actions, metadata, schema changes
4. **Enables** ACID transactions, time travel, audit trail

### **How It Works**

* **Reads** - Replay transaction log to find current files
* **Writes** - Add data files, then atomically commit transaction
* **Versions** - Each transaction creates a new version
* **Snapshots** - Each version is a consistent snapshot

### **ACID Properties**

* **Atomicity** - Transaction log entry = atomic commit
* **Consistency** - Only valid transactions recorded
* **Isolation** - Version numbers prevent conflicts
* **Durability** - Cloud storage durability

### **Time Travel**

* **Query any version** - VERSION AS OF or TIMESTAMP AS OF
* **Compare versions** - Audit changes
* **Restore versions** - Rollback mistakes
* **Limited by VACUUM** - Old versions eventually deleted

### **In Serverless**

* ✅ Use DESCRIBE HISTORY (no file access needed)
* ✅ Use time travel queries
* ✅ Use RESTORE command
* ❌ Cannot directly access _delta_log files
* ✅ All features work through SQL/Python APIs

## 🎉 Congratulations!

You've completed the Delta Lake Transaction Logs demo!

### **What You Learned:**

✅ **Transaction Log Architecture** - How Delta Lake tracks changes  
✅ **ACID Properties** - Atomicity, Consistency, Isolation, Durability  
✅ **DESCRIBE HISTORY** - View transaction log without file access  
✅ **Time Travel** - Query and restore historical versions  
✅ **Practical Patterns** - Audit trails, debugging, rollback  
✅ **Best Practices** - Retention, monitoring, performance  

---

### **Key Takeaways:**

1. **Transaction log is the foundation** - Enables all Delta Lake features
2. **Every operation creates a version** - Complete audit trail
3. **ACID guarantees** - Reliable data operations
4. **Time travel is powerful** - Query any historical version
5. **Works in serverless** - No file system access needed
6. **Balance retention vs cost** - Use VACUUM appropriately

---

### **The Magic of Delta Lake:**

```
Parquet files (data) + Transaction Log (metadata) = Delta Lake ✨
```

The transaction log transforms simple Parquet files into a powerful, ACID-compliant data lake!

---

### **Next Steps:**

* Explore MERGE operations (upserts)
* Learn about Delta Lake optimization (OPTIMIZE, Z-ORDER)
* Study Change Data Feed (CDC)
* Implement data retention policies
* Build production pipelines with Delta Lake

---

### **Resources:**

* [Delta Lake Documentation](https://docs.databricks.com/delta/index.html)
* [Transaction Log Protocol](https://github.com/delta-io/delta/blob/master/PROTOCOL.md)
* [Time Travel Guide](https://docs.databricks.com/delta/history.html)
* [Delta Lake Best Practices](https://docs.databricks.com/delta/best-practices.html)

---

**You now understand the foundation of Delta Lake!** 🚀

*Happy data engineering!*